In [1]:
import sympy as smp
import numpy as np
import matplotlib.pyplot as plt

Pandas
Statsmodels
itertools

In [2]:
# smp.eye()
# smp.Array([])
# smp.tensorproduct(Tensor 1, Tensor 2)
# smp.tensorcontraction(Tensor, (first index, second index))

# <center>Defining Metrics Tensors, Energy-Momentum Tensors and Variables<center>

## The Variables

In [3]:
alpha, beta, r_s, G, M, t, r, theta, phi, tau, x, y, z = smp.symbols('alpha beta r_s G M t r theta phi tau x y z', float = True)

## The Metric Tensors

#### The Schwarzschild Metric

In [4]:
t_p, r_p, theta_p, phi_p = smp.symbols('t_p r_p theta_p phi_p', cls = smp.Function)

t_p = t_p(tau)
r_p = r_p(tau)
theta_p = theta_p(tau)
phi_p = phi_p(tau)

Schild = { 'Metric' : smp.MutableDenseNDimArray([[-(1 - (r_s)/(r_p)),0,0,0],
                                                 [0,1/(1 - (r_s)/(r_p)),0,0],
                                                 [0,0,r_p**2,0],
                                                 [0,0,0,r_p**2*smp.sin(theta_p)**2]]),
          'Basis' : smp.Array([t_p,r_p,theta_p,phi_p]),
          
          'Dimention' : 4, 
          
          'Basis_Parameter' : smp.MutableDenseNDimArray([t_p, r_p, theta_p, phi_p]) }

#### The Spherical Polar Metric

In [5]:
Polar_Coordinates = { 'Metric' : smp.MutableDenseNDimArray([[1, 0, 0], [0, r**2, 0], [0, 0, r**2*smp.sin(theta)**2]]),
                       
                       'Basis' : smp.Array([r, theta , phi]),
                     
                     'Dimention' : 3}

#### A 2-Sphere Metric

In [6]:
Two_Sphere = { 'Metric' : smp.MutableDenseNDimArray([[r**2, 0], [0, r**2*smp.sin(theta)**2]]) ,
            
               'Basis' : smp.Array([theta, phi]) , 
              
              'Dimention' : 2}

#### A General Spherically Symmetric Metric

In [7]:
t_p, r_p, theta_p, phi_p = smp.symbols('t_p r_p theta_p phi_p', cls = smp.Function)

t_p = t_p(tau)
r_p = r_p(tau)
theta_p = theta_p(tau)
phi_p = phi_p(tau)

A , B = smp.symbols('A B', cls = smp.Function)
A = A(r_p)
B = B(r_p)


Sphecally_Symmetric_Metric = { 'Metric' : smp.MutableDenseNDimArray([[-A,0,0,0],
                                                                     [0,B,0,0],
                                                                     [0,0,r_p**2,0],
                                                                     [0,0,0,r_p**2*smp.sin(theta_p)**2]]),
                              
                              'Basis' : smp.Array([t_p,r_p,theta_p,phi_p]),
                              
                              'Dimention' : 4,
                              
                              'Basis_Parameter' : smp.MutableDenseNDimArray([t_p, r_p, theta_p, phi_p])}

# <center>Functions of the Objects from Differential Geometry<center>

In general relativity, either:

1. We know the metric to begin with. (From which we can calculate anything we like with it using the functions bellow.)

2. We do not know the metric, but wish to find it. (In which case we must solve Einsteins equations using the functions bellow.)

### Derivative of Rank-2 Tensor

$$\partial_{\mu} g_{\nu \beta}$$

In [8]:
def Derivative(T):
    G = T['Metric']
    N = T['Dimention']
    basis = T['Basis']
    A = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                A[i,j,k] = smp.diff(G[j,k],basis[i])
    return A

### Inverse of Rank-2 Tensor

$$ g^{\nu \beta} = (g^{-1})_{\nu \beta}$$

In [9]:
def inv(T):
    G = T['Metric']
    N = T['Dimention']
    basis = T['Basis']
    g_m = G.tomatrix()
    inv_g = g_m.inv()
    A = smp.MutableDenseNDimArray(smp.zeros(N**2),(N,N))
    for i in range(N):
        for j in range(N):
            A[i,j] = inv_g[i, j]
    return A

### The Christoffel Symbols

$$ \Gamma^{\alpha}_{\mu \nu} = \frac{1}{2} g^{\alpha \beta} \left( \partial_{\mu} g_{\nu \beta} + \partial_{\nu} g_{\mu \beta} - \partial_{\beta} g_{\mu \nu} \right) $$

In [70]:
def Gamma(T):
    G = T['Metric']
    N = T['Dimention']
    basis = T['Basis']
    A1 = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
    ig = inv(T)
    for i in range(N):
        for j in range(N):
            for k in range(N):
                for d in range(N):
                    A1[i, j, k] += smp.Rational(1, 2)*(ig[d,i])*(smp.diff(G[k,d],basis[j]) + smp.diff(G[d,j],basis[k]) - smp.diff(G[j,k],basis[d]))
            
    
    return A1

In [71]:
Gamma(Schild)

[[[0, -r_s/(2*(r_s/r_p(tau) - 1)*r_p(tau)**2), 0, 0], [-r_s/(2*(r_s/r_p(tau) - 1)*r_p(tau)**2), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[r_s*(-r_s/(2*r_p(tau)) + 1/2)/r_p(tau)**2, 0, 0, 0], [0, -r_s*(-r_s/(2*r_p(tau)) + 1/2)/((-r_s/r_p(tau) + 1)**2*r_p(tau)**2), 0, 0], [0, 0, -2*(-r_s/(2*r_p(tau)) + 1/2)*r_p(tau), 0], [0, 0, 0, -2*(-r_s/(2*r_p(tau)) + 1/2)*r_p(tau)*sin(theta_p(tau))**2]], [[0, 0, 0, 0], [0, 0, 1/r_p(tau), 0], [0, 1/r_p(tau), 0, 0], [0, 0, 0, -sin(theta_p(tau))*cos(theta_p(tau))]], [[0, 0, 0, 0], [0, 0, 0, 1/r_p(tau)], [0, 0, 0, cos(theta_p(tau))/sin(theta_p(tau))], [0, 1/r_p(tau), cos(theta_p(tau))/sin(theta_p(tau)), 0]]]

### The Covarient Derivatives

Acting on: The Contravariant (1,0) Tensor (Vector) Field 

$$D_{\mu}V^{\nu} = {\partial}_{\mu}V^{\nu} + {\Gamma}^{\nu}_{\mu \alpha}V^{\alpha}$$

In [11]:
def CovariantD10(T, V):
    N = T['Dimention']
    basis = T['Basis']
    C = Gamma(T)
    DV = smp.MutableDenseNDimArray(smp.zeros(N**2),(N,N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                DV[i,j] += smp.Rational(1, N)*smp.diff(V[j],basis[i]) + C[j,i,k]*V[k]
    return DV

Acting on: The Covariant (0,1) Tensor (Vector) Field

$$D_{\mu}V_{\nu} = {\partial}_{\mu}V_{\nu} - {\Gamma}^{\alpha}_{\mu \nu}V_{\alpha}$$

In [12]:
def CovariantD01(T, V):
    N = T['Dimention']
    basis = T['Basis']
    C = Gamma(T)
    DV = smp.MutableDenseNDimArray(smp.zeros(N**2),(N,N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                DV[i,j] += smp.Rational(1, N)*smp.diff(V[j],basis[i]) - C[k,i,j]*V[k]
    return DV

Acting on: Type (2,0) Tensor Field

$$D_{\alpha}T^{\mu \nu} = {\partial}_{\alpha}T^{\mu \nu} + {\Gamma}^{\mu}_{\alpha \gamma}T^{\gamma \nu} + {\Gamma}^{\nu}_{\alpha \gamma}T^{\gamma \mu}$$

In [13]:
def CovariantD20(G, T):
    N = G['Dimention']
    basis = G['Basis']
    C = Gamma(G)
    DT = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                for p in range(N):
                    DT[i,j,k] += smp.Rational(1, N)*smp.diff(T[j,k],basis[i]) + C[j,i,p]*T[p,k] + C[k,i,p]*T[p,j]
    return DT

Acting on: Type (0,2) Tensor Field

$$D_{\alpha}T_{\mu \nu} = {\partial}_{\alpha}T_{\mu \nu} - {\Gamma}^{\gamma}_{\alpha \mu}T_{\gamma \nu} - {\Gamma}^{\gamma}_{\alpha \nu}T_{\gamma \mu}$$

In [14]:
def CovariantD02(G, T):
    N = G['Dimention']
    basis = G['Basis']
    C = Gamma(G)
    DT = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                for p in range(N):
                    DT[i,j,k] += smp.Rational(1, N)*smp.diff(T[j,k],basis[i]) - C[p,i,j]*T[p,k] - C[p,i,k]*T[p,j]
    return DT

Acting on: Type (1,1) Tensor Field

$$D_{\alpha}T^{\mu}_{\nu} = {\partial}_{\alpha}T^{\mu}_{\nu} + {\Gamma}^{\mu}_{\alpha \gamma}T^{\gamma}_{\nu} - {\Gamma}^{\gamma}_{\alpha \nu}T^{\mu}_{\gamma}$$

In [15]:
def CovariantD11(G, T):
    N = G['Dimention']
    basis = G['Basis']
    C = Gamma(G)
    DT = smp.MutableDenseNDimArray(smp.zeros(N**3),(N,N,N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                for p in range(N):
                    DT[i,j,k] += smp.Rational(1, N)*smp.diff(T[j,k],basis[i]) + C[j,i,p]*T[p,k] - C[p,i,k]*T[j,p]
    return DT

### The Riemannm Tensor

$$ R^{ \rho}_{\sigma \mu \nu}  = {\partial}_{\mu} {\Gamma}^{\rho}_{\nu \sigma} - {\partial}_{\nu} {\Gamma}^{\rho}_{\mu \sigma} + {\Gamma}^{\rho}_{\mu \lambda}{\Gamma}^{\lambda}_{\nu \sigma} - {\Gamma}^{\rho}_{\nu \lambda}{\Gamma}^{\lambda}_{\mu \sigma} $$

In [72]:
def Riemann(M):
    T = M['Metric']
    N = M['Dimention']
    basis = M['Basis']
    G = Gamma(M)
    R = smp.MutableDenseNDimArray(smp.zeros(N**4),(N,N,N,N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                for p in range(N):
                    for d in range(N):
                        R[i, j, k, p] += smp.Rational(1, N)*(smp.diff(G[i,p,j],basis[k])-
                                                             smp.diff(G[i,k,j],basis[p]))+(G[i,k,d]*G[d,p,j]-G[i,p,d]*G[d,k,j])

    
    return R

In [73]:
Riemann(Schild)

[[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, r_s**2/(4*(r_s/r_p(tau) - 1)**2*r_p(tau)**4) + r_s**2*(-r_s/(2*r_p(tau)) + 1/2)/(2*(-r_s/r_p(tau) + 1)**2*(r_s/r_p(tau) - 1)*r_p(tau)**4) - r_s/((r_s/r_p(tau) - 1)*r_p(tau)**3), 0, 0], [-r_s**2/(4*(r_s/r_p(tau) - 1)**2*r_p(tau)**4) - r_s**2*(-r_s/(2*r_p(tau)) + 1/2)/(2*(-r_s/r_p(tau) + 1)**2*(r_s/r_p(tau) - 1)*r_p(tau)**4) + r_s/((r_s/r_p(tau) - 1)*r_p(tau)**3), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, r_s*(-r_s/(2*r_p(tau)) + 1/2)/((r_s/r_p(tau) - 1)*r_p(tau)), 0], [0, 0, 0, 0], [-r_s*(-r_s/(2*r_p(tau)) + 1/2)/((r_s/r_p(tau) - 1)*r_p(tau)), 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, r_s*(-r_s/(2*r_p(tau)) + 1/2)*sin(theta_p(tau))**2/((r_s/r_p(tau) - 1)*r_p(tau))], [0, 0, 0, 0], [0, 0, 0, 0], [-r_s*(-r_s/(2*r_p(tau)) + 1/2)*sin(theta_p(tau))**2/((r_s/r_p(tau) - 1)*r_p(tau)), 0, 0, 0]]], [[[0, -r_s**2*(-r_s/(2*r_p(tau)) + 1/2)/(2*(r_s/r_p(tau) - 1)*r_p(tau)**4) - r_s**2/(2*r_p(tau)**4) + r_s**2*(-r_s/(2*r_p(tau)) + 1/2)**2/

Purely covariant Riemann tensor:
    
$$ R_{\rho \sigma \mu \nu} = g_{\rho \alpha}R^{\alpha}_{\sigma \mu \nu}$$

In [17]:
def CovRiemann(T):
    G = T['Metric']
    N = T['Dimention']
    basis = T['Basis']
    R = Riemann(T)
    RL = smp.MutableDenseNDimArray(smp.zeros(N**4),(N,N,N,N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                for p in range(N):
                    for d in range(N):
                        RL[i,j,k,p] += G[i,d]*R[d,j,k,p]
    return RL

### The Ricci Tensor

$$ R_{\mu \nu} = g^{\alpha \beta} R_{\alpha \mu \beta \nu}$$

In [18]:
def Ricci(T):
    G = T['Metric']
    N = T['Dimention']
    basis = T['Basis']
    ig = inv(T)
    RL = CovRiemann(T)
    Ric = smp.MutableDenseNDimArray(smp.zeros(N**2),(N,N))
    for i in range(N):
        for j in range(N):
            for d in range(N):
                for s in range(N):
                    Ric[i,j] += ig[d,s]*RL[d,i,s,j]
    return Ric

### The Ricci Scalar

$$ R = g^{\mu \nu}R_{\mu \nu}$$

In [97]:
def RiccScalar(T):
    G = T['Metric']
    N = T['Dimention']
    basis = T['Basis']
    R = Ricci(T)
    ig = inv(T)
    S = float()
    for d in range(N):
        for s in range(N):
            S = ig[d,s]*R[d,s]
    return S

### The Curvature Scalar

$$ S = g^{\sigma \alpha}g^{\rho \beta}g^{\mu \gamma}g^{\epsilon \nu}R_{\alpha \beta \mu \nu} R_{\sigma \rho \gamma \epsilon} = R^{\sigma \rho \gamma \epsilon}R_{\sigma \rho \gamma \epsilon}$$

In [94]:
def CurvScalar(T):
    G = T['Metric']
    N = T['Dimention']
    basis = T['Basis']
    R = CovRiemann(T)
    ig = inv(T)
    CS = float()
    for i in range(N):
        for j in range(N):
            for k in range(N):
                for p in range(N):
                    for d in range(N):
                        for n in range(N):
                            for s in range(N):
                                for t in range(N):
                                    CS += ig[i,j]*ig[k,p]*ig[d,n]*ig[s,t]*R[i,k,d,s]*R[j,p,d,s]
    return CS

In [96]:
CurvScalar(Schild).simplify()

12*r_s**2/r_p(tau)**6

In [ ]:
for i in range(4):
    for j in range(4):

### The Geodesic Equation

$$ \frac{d^2x^{\mu}}{d\tau^2} + {\Gamma}^{\mu}_{\alpha \beta} \frac{dx^{\alpha}}{d\tau}\frac{dx^{\beta}}{d\tau} = 0$$

In [21]:
def Geodesic(T):
    N = T['Dimention']
    basis = T['Basis']
    P = T['Basis_Parameter']
    C = Gamma(T)
    Geo = smp.MutableDenseNDimArray(smp.zeros(N),(N))
    for i in range(N):
        for j in range(N):
            for k in range(N):
                Geo[[i]] += smp.Rational(1, N**2)*(smp.diff(basis[i], tau, tau)) + C[i, j, k]*(smp.diff(basis[j], tau))*(smp.diff(basis[k], tau))
    
    return Geo

### The Euler-Lagrange Equation in GR

<b> First we define the Lagrangian above:
    

$$ L = \sqrt{-g_{\mu \nu}\frac{dx^{\mu}}{d\tau}\frac{dx^{\nu}}{d\tau}} $$

In [22]:
def Lagrangian(T):
    G = T['Metric']
    N = T['Dimention']
    basis = T['Basis']
    P = T['Basis_Parameter']
    L = float()
    for i in range(N):
        for j in range(N):
            L += - G[i,j]*smp.diff(P[i], tau)*smp.diff(P[j], tau)
    
    return smp.sqrt(L)

<b> Second, we define the Euler-Lagrange Equations:
    
$$ \frac{\partial L}{\partial x^{\alpha}} - \frac{d}{d\tau}\frac{\partial L}{\partial \frac{\partial x^{\alpha}}{\partial \tau}}=0$$
    
Note: To greatly simplify these equations, the code sets the lagrangian equal to 1 in each iteration of calculation. We are allowed to do this because we can arbitrarily set the length of the geodesic equation to be what we like. The path itself is what we are interested in, the length of the path is really just not so important. All we are doing by setting $L=1$ at the end of our calculations is setting the length of our geodesic curve, parametrixzed by $\tau$ to be $length = 1$ 

In [23]:
def Euler_Lagrange(L, T):
    N = T['Dimention']
    basis = T['Basis']
    P = T['Basis_Parameter']
    
    dpdt = smp.MutableDenseNDimArray(smp.zeros(N),(N))
    for i in range(N):
        dpdt[[i]] = smp.diff(P[i],tau)
    
    A = smp.MutableDenseNDimArray(smp.zeros(N),(N))
    B = smp.MutableDenseNDimArray(smp.zeros(N),(N))
    for i in range(N):
        B[[i]] = smp.diff(L, dpdt[i])
    for i in range(N):
        A[[i]] = B[i].subs(L, 1)
    
    
    C = smp.MutableDenseNDimArray(smp.zeros(N),(N))
    D = smp.MutableDenseNDimArray(smp.zeros(N),(N))
    for i in range(N):
        C[[i]] = smp.diff(L, P[i])
    for i in range(N):
        D[[i]] = C[i].subs(L, 1)
    
    La = smp.MutableDenseNDimArray(smp.zeros(N),(N))
    for i in range(N):
            La[[i]] = D[i] - smp.diff(A[i], tau)
    
    return La

### The Einstein Field Equations

# <center>Calculating the Schwarzschild solution<center>

In [31]:
eq_1 = Ricci(Sphecally_Symmetric_Metric)[0,0].simplify()
eq_2 = Ricci(Sphecally_Symmetric_Metric)[1,1].simplify()
eq_3 = Ricci(Sphecally_Symmetric_Metric)[2,2].simplify()
eq_4 = Ricci(Sphecally_Symmetric_Metric)[3,3].simplify()

In [32]:
eq_1

Derivative(A(r_p(tau)), (r_p(tau), 2))/(2*B(r_p(tau))) + Derivative(A(r_p(tau)), r_p(tau))/(B(r_p(tau))*r_p(tau)) - Derivative(A(r_p(tau)), r_p(tau))*Derivative(B(r_p(tau)), r_p(tau))/(4*B(r_p(tau))**2) - Derivative(A(r_p(tau)), r_p(tau))**2/(4*A(r_p(tau))*B(r_p(tau)))

In [33]:
eq_2

Derivative(B(r_p(tau)), r_p(tau))/(B(r_p(tau))*r_p(tau)) - Derivative(A(r_p(tau)), (r_p(tau), 2))/(2*A(r_p(tau))) + Derivative(A(r_p(tau)), r_p(tau))*Derivative(B(r_p(tau)), r_p(tau))/(4*A(r_p(tau))*B(r_p(tau))) + Derivative(A(r_p(tau)), r_p(tau))**2/(4*A(r_p(tau))**2)

In [34]:
eq_3

1 - 1/B(r_p(tau)) + r_p(tau)*Derivative(B(r_p(tau)), r_p(tau))/(2*B(r_p(tau))**2) - r_p(tau)*Derivative(A(r_p(tau)), r_p(tau))/(2*A(r_p(tau))*B(r_p(tau)))

In [35]:
eq_4

sin(theta_p(tau))**2 - sin(theta_p(tau))**2/B(r_p(tau)) + r_p(tau)*sin(theta_p(tau))**2*Derivative(B(r_p(tau)), r_p(tau))/(2*B(r_p(tau))**2) - r_p(tau)*sin(theta_p(tau))**2*Derivative(A(r_p(tau)), r_p(tau))/(2*A(r_p(tau))*B(r_p(tau)))

In [36]:
eq_5 = ((eq_1*B + eq_2*A)*(r*B)).simplify()

In [37]:
A1 = smp.dsolve(eq_5, B)

AssertionError: 

In [38]:
A1

NameError: name 'A1' is not defined

In [39]:
eq_6 = eq_3.subs([(B, 1/A)]).simplify()

In [40]:
eq_6

-A(r_p(tau)) - r_p(tau)*Derivative(A(r_p(tau)), r_p(tau)) + 1

In [41]:
A0 = smp.dsolve(eq_6, A)

AssertionError: 

Finnaly we have a solution to what we want. We find:

$$ A(r) = 1 + \frac{C_{0}}{r}$$
    


In [42]:
C = smp.symbols('C')
A = 1 + C/r

In [43]:
smp.limit(A, r, smp.oo)

1

This is what we want! As we go infinetly far from the Black Hole, we get to a flat geometry of spacetime.

# <center>Calculating Orbital Equations<center>

In curved spaces, particles follow paths known as geodesics, where massive and massless particles do, of course, have slightly different geodesics. Given we now want to describe moving particles in curved spaces, geodesic equations are indispensable. There are a few different ways the geodesic can be derived, an elegant method is by extremizing the proper time:

$$ \tau_{AB} = \int^{A}_{B} \sqrt{-g_{\mu \nu}\frac{dx^{\mu}}{d\tau}\frac{dx^{\nu}}{d\tau}} $$

A to B is the proper time A
elapsed in space-time and we will find it convenient later to make it a unit length.

The Lagrangian equation we wish to miimise is given by:

In [44]:
L = Lagrangian(Sphecally_Symmetric_Metric)
L

sqrt(A(r_p(tau))*Derivative(t_p(tau), tau)**2 - B(r_p(tau))*Derivative(r_p(tau), tau)**2 - r_p(tau)**2*sin(theta_p(tau))**2*Derivative(phi_p(tau), tau)**2 - r_p(tau)**2*Derivative(theta_p(tau), tau)**2)

By setting $L=1$ we find an extra equation of motion given by:

In [45]:
E1 = 1 - Lagrangian(Sphecally_Symmetric_Metric)**2
E1

-A(r_p(tau))*Derivative(t_p(tau), tau)**2 + B(r_p(tau))*Derivative(r_p(tau), tau)**2 + r_p(tau)**2*sin(theta_p(tau))**2*Derivative(phi_p(tau), tau)**2 + r_p(tau)**2*Derivative(theta_p(tau), tau)**2 + 1

We now wish to solve the equation of motions, by substituting our lagrangian above into the Euler-Lagrange Equations:

$$ \frac{\partial L}{\partial x^{\alpha}} - \frac{d}{d\tau}\frac{\partial L}{\partial \frac{\partial x^{\alpha}}{\partial \tau}}=0$$

We have a 4-Dimentional spacetime and therefore there will be 4 differential equations:

1. The Euler-Lagrange for $t$
2. The Euler-Lagrange for $r$
3. The Euler-Lagrange for $\theta$
4. The Euler-Lagrange for $\phi$
5. (We also have the extra equation givan above for $L=1$

In [46]:
Et = Euler_Lagrange(L, Sphecally_Symmetric_Metric)[0]
Et

-A(r_p(tau))*Derivative(t_p(tau), (tau, 2)) - Derivative(A(r_p(tau)), r_p(tau))*Derivative(r_p(tau), tau)*Derivative(t_p(tau), tau)

In [47]:
Er = Euler_Lagrange(L, Sphecally_Symmetric_Metric)[1]
Er

B(r_p(tau))*Derivative(r_p(tau), (tau, 2)) - r_p(tau)*sin(theta_p(tau))**2*Derivative(phi_p(tau), tau)**2 - r_p(tau)*Derivative(theta_p(tau), tau)**2 + Derivative(A(r_p(tau)), r_p(tau))*Derivative(t_p(tau), tau)**2/2 + Derivative(B(r_p(tau)), r_p(tau))*Derivative(r_p(tau), tau)**2/2

In [48]:
Etheta = Euler_Lagrange(L, Sphecally_Symmetric_Metric)[2]
Etheta

-r_p(tau)**2*sin(theta_p(tau))*cos(theta_p(tau))*Derivative(phi_p(tau), tau)**2 + r_p(tau)**2*Derivative(theta_p(tau), (tau, 2)) + 2*r_p(tau)*Derivative(r_p(tau), tau)*Derivative(theta_p(tau), tau)

In [49]:
Ephi = Euler_Lagrange(L, Sphecally_Symmetric_Metric)[3]
Ephi

r_p(tau)**2*sin(theta_p(tau))**2*Derivative(phi_p(tau), (tau, 2)) + 2*r_p(tau)**2*sin(theta_p(tau))*cos(theta_p(tau))*Derivative(phi_p(tau), tau)*Derivative(theta_p(tau), tau) + 2*r_p(tau)*sin(theta_p(tau))**2*Derivative(phi_p(tau), tau)*Derivative(r_p(tau), tau)

Before continue with the maths let’s think about some physics. Firstly, due to conservation of angular momentum, we know planets orbit on fixed planes, this can be seen from the independence of $\phi$ in our Lagrangian $L$. Because of this, we can set $\theta$ to a constant of our choice, this choice will represent the angle of the plane which our orbital mechanics are taking place. For simplicity we will choose $\theta = \frac{\pi}{2}$, consequently $sin(\frac{\pi}{2}) = 1$ and  $\frac{d\theta}{d\tau}=0$.

In [50]:
Er.subs(theta_p , smp.pi/2).simplify()

B(r_p(tau))*Derivative(r_p(tau), (tau, 2)) - r_p(tau)*Derivative(phi_p(tau), tau)**2 + Derivative(A(r_p(tau)), r_p(tau))*Derivative(t_p(tau), tau)**2/2 + Derivative(B(r_p(tau)), r_p(tau))*Derivative(r_p(tau), tau)**2/2

In [51]:
Etheta.subs(theta_p , smp.pi/2).simplify()

0

In [52]:
Ephi.subs(theta_p , smp.pi/2).simplify()

(r_p(tau)*Derivative(phi_p(tau), (tau, 2)) + 2*Derivative(phi_p(tau), tau)*Derivative(r_p(tau), tau))*r_p(tau)

In [53]:
E2 = E1.subs(theta_p , smp.pi/2).simplify()

Let us find an expression for $t(\tau)$ then substitute it into $E1$

In [54]:
St = smp.dsolve(Et, t_p)
St

Eq(t_p(tau), C1 + C2*Integral(1/A(r_p(tau)), tau))

From this equation we can see that the following differential equation holds:

In [55]:
smp.diff(t_p, tau) - alpha/A

-alpha/(C/r + 1) + Derivative(t_p(tau), tau)

We can substitute this into our new Lagrangian E2:

In [56]:
E3 = E2.subs(smp.diff(t_p,tau),alpha/A)
E3

-alpha**2*A(r_p(tau))/(C/r + 1)**2 + B(r_p(tau))*Derivative(r_p(tau), tau)**2 + r_p(tau)**2*Derivative(phi_p(tau), tau)**2 + 1

All that is left is some substitution for $\phi$, then we will have an equation of motion for $r$ as a function of some parameter $\tau$

In [57]:
phi = smp.dsolve(Ephi.subs(theta_p , smp.pi/2).simplify(), phi_p)
phi

Eq(phi_p(tau), C1 + C2*Integral(r_p(tau)**(-2), tau))

We can simply differentiate this once w.r.t $\tau$ to find the equation we will substitute into the Lagrangian, now called $E3$

In [58]:
smp.diff(phi_p, tau) - beta/(r_p)**2

-beta/r_p(tau)**2 + Derivative(phi_p(tau), tau)

In [59]:
E4 = E3.subs(smp.diff(phi_p,tau),beta/(r_p)**2)
E4

-alpha**2*A(r_p(tau))/(C/r + 1)**2 + beta**2/r_p(tau)**2 + B(r_p(tau))*Derivative(r_p(tau), tau)**2 + 1

In [60]:
(E4/(2*B)).simplify()

-alpha**2*r**2*A(r_p(tau))/(2*(C + r)**2*B(r_p(tau))) + beta**2/(2*B(r_p(tau))*r_p(tau)**2) + Derivative(r_p(tau), tau)**2/2 + 1/(2*B(r_p(tau)))

In [61]:
phi, M, l, eta = smp.symbols('phi M l eta')
y = smp.Function('y')
y = y(phi)

In [62]:
DE = smp.diff(y, phi, phi) + y - M/(l**2) - ((y**2)*eta*l**2)/M

In [63]:
DE1 = smp.diff(y, phi, phi) + y - M/(l**2)

In [64]:
DE1

-M/l**2 + y(phi) + Derivative(y(phi), (phi, 2))

In [65]:
smp.dsolve(DE1, y, x0 = 0, n=0)

Eq(y(phi), C1*sin(phi) + C2*cos(phi) + M/l**2)

In [66]:
smp.dsolve(DE1, y)

Eq(y(phi), C1*sin(phi) + C2*cos(phi) + M/l**2)

In [67]:
y.series(y, 0, 3)

y(phi)

In [68]:
ep = smp.symbols('alpha')
y_0, y_1, y_2, y_3 = smp.symbols('y_0 y_1 y_2 y_3', cls = smp.Function)
y_0 = y_0(phi)
y_1 = y_1(phi)
y_2 = y_2(phi)
y_3 = y_3(phi)
sy = y_0 + ep*y_1 + ep**2*y_2

In [69]:
DE.subs(y, sy).simplify()

-M/l**2 + alpha**2*y_2(phi) + alpha**2*Derivative(y_2(phi), (phi, 2)) + alpha*y_1(phi) + alpha*Derivative(y_1(phi), (phi, 2)) + y_0(phi) + Derivative(y_0(phi), (phi, 2)) - eta*l**2*(alpha**2*y_2(phi) + alpha*y_1(phi) + y_0(phi))**2/M

In [78]:
x = smp.symbols('x')
y = smp.symbols('y', cls = smp.Function)
y = y(x)

In [82]:
y = smp.sin(x**10)

In [83]:
y

sin(x**10)

In [86]:
b = smp.diff(y, x)
b

10*x**9*cos(x**10)

In [93]:
b.subs(x, 10)

10000000000*cos(10000000000)

In [ ]:
smp.e